In [1]:
import PIL
from torchvision import transforms

def get_test_img(file):
    test_img = PIL.Image.open(f'test pictures\\{file}')
    return transforms.ToTensor()(test_img)

In [4]:
import torch
import os
from models.SR3Builder import SR3Builder
from models.SRDIFFBuilder import SRDiffBuilder
from utils.model_utils import load_model

sr3_models = os.listdir('saved models\\SR3')
device = torch.device(0)
model_builder = SR3Builder()
model = model_builder.set_standart().build()
model.to(device)
sr_images = []
for sr3_model in sr3_models:
    try:
        torch.cuda.empty_cache()
        model = load_model(model, sr3_model, 'saved models\\SR3\\')
        bicubic = get_test_img("Amnesty POI-6-3-1.png").unsqueeze(0).to(device)
        lr = get_test_img("Amnesty POI-6-3-1-lr.png").unsqueeze(0).to(device)    
        sr = model.sample(bicubic)
        torch.cuda.empty_cache()
        sr_images.append(sr.squeeze().to('cpu').detach())
    except Exception as e:
        print(sr3_model, e)
        continue

The model weights have been loaded from 'saved models\SR3\SR3 standart 128 bach DA version 1.pt'
The model weights have been loaded from 'saved models\SR3\SR3 Standart 128 bach DA.pt'
The model weights have been loaded from 'saved models\SR3\SR3 Standart 128 bach.pt'
The model weights have been loaded from 'saved models\SR3\SR3 standart 32 bach DA version 1.pt'
The model weights have been loaded from 'saved models\SR3\SR3 Standart 32 bach DA.pt'
The model weights have been loaded from 'saved models\SR3\SR3 Standart 32 bach.pt'
The model weights have been loaded from 'saved models\SR3\SR3 standart 64 bach DA version 1.pt'
The model weights have been loaded from 'saved models\SR3\SR3 Standart 64 bach DA.pt'
The model weights have been loaded from 'saved models\SR3\SR3 Standart 64 bach.pt'
The model weights have been loaded from 'saved models\SR3\SR3 Standart DA.pt'
The model weights have been loaded from 'saved models\SR3\SR3 Standart l1 DA 128b.pt'
The model weights have been loaded fro

In [5]:
from utils.tensor_utils import tensor2img

for idx, tensor in enumerate(sr_images):
    img_np = tensor2img(tensor)
    img = PIL.Image.fromarray(img_np)
    img.save(f"test pictures\\SR3\\{sr3_models[idx]}.png")